In [1]:
# --- Cell 1: Imports ---
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm
import random

print(f"Imports loaded. Albumentations version: {A.__version__}")

Imports loaded. Albumentations version: 1.3.1


In [2]:
# --- Cell 2: Setup Paths and Parameters ---

# Path to the 25 example images
image_dir = r'C:\Users\dadab\Desktop\AUG paper\Split_Dataset\examples'

# Path to save the stacked comparison images
output_dir = r'C:\Users\dadab\Desktop\AUG paper\aug_visual_results'

# BGR mean value from the config (for fill)
mean_val = [123.675, 116.28, 103.53]

# The standard 'Resize' operation from the baseline
# Replicates mmdet's Resize(scale=(1333, 800), keep_ratio=True)
resize_op = A.LongestMaxSize(max_size=1333, interpolation=cv2.INTER_LINEAR)

print("Paths and parameters set.")

Paths and parameters set.


In [3]:
# --- Cell 3: Helper Function ---

def create_and_save_comparison(original_img, augmented_img, folder_path, filename):
    """
    Stacks an original and augmented image vertically and saves the result.
    Original (top), Augmented (bottom).
    """
    try:
        # Ensure images are 8-bit unsigned integers
        original_img = original_img.astype(np.uint8)
        augmented_img = augmented_img.astype(np.uint8)

        # Get augmented image dimensions
        h_aug, w_aug, _ = augmented_img.shape
        
        # Resize original image to the *exact* size as the augmented image
        original_resized = cv2.resize(original_img, (w_aug, h_aug), interpolation=cv2.INTER_LINEAR)

        # Stack them vertically
        comparison_image = cv2.vconcat([original_resized, augmented_img])
        
        # Create the destination folder if it doesn't exist
        os.makedirs(folder_path, exist_ok=True)
        
        # Save the final comparison image
        save_path = os.path.join(folder_path, filename)
        cv2.imwrite(save_path, comparison_image)
        
    except Exception as e:
        print(f"Error processing {filename}: {e}")

print("Helper function defined.")

Helper function defined.


In [4]:
# --- Cell 4: Load Images ---

original_images = []
# List all files in the image directory
image_filenames = os.listdir(image_dir)

for img_file in image_filenames:
    # Ensure the file is an image
    if img_file.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(image_dir, img_file)
        # Read the image using OpenCV
        img = cv2.imread(img_path)
        if img is not None:
            # Store the image and its filename
            original_images.append({'img': img, 'filename': img_file})

print(f"Loaded {len(original_images)} images from {image_dir}")

Loaded 25 images from C:\Users\dadab\Desktop\AUG paper\Split_Dataset\examples


In [5]:
# --- Cell 5: Manual MixUp/CutMix Implementations (NumPy/CV2) ---
# These functions replicate the MMDetection logic without using MMDetection.

def mixup_data(img1, img2, alpha=1.0):
    """Applies MixUp to two images of the same size."""
    # Generate a blending factor from a Beta distribution
    lam = np.random.beta(alpha, alpha)
    mixed_img = lam * img1 + (1 - lam) * img2
    return mixed_img.astype(np.uint8)

def cutmix_data(img1, img2, alpha=1.0):
    """Applies CutMix to two images of the same size."""
    h, w, _ = img1.shape
    lam = np.random.beta(alpha, alpha)
    
    # Calculate box coordinates based on lambda
    r_x = np.random.uniform(0, w)
    r_y = np.random.uniform(0, h)
    r_w = w * np.sqrt(1 - lam)
    r_h = h * np.sqrt(1 - lam)
    x1 = int(np.round(max(r_x - r_w / 2, 0)))
    y1 = int(np.round(max(r_y - r_h / 2, 0)))
    x2 = int(np.round(min(r_x + r_w / 2, w)))
    y2 = int(np.round(min(r_y + r_h / 2, h)))

    # Create a copy and paste the patch from img2 onto img1
    img1_cut = img1.copy()
    img1_cut[y1:y2, x1:x2, :] = img2[y1:y2, x1:x2, :]
    return img1_cut.astype(np.uint8)

def pad_to_match(img1, img2, pad_val):
    """Pads two images (H, W, C) to the maximum H/W dimensions."""
    h1, w1, _ = img1.shape
    h2, w2, _ = img2.shape
    max_h = max(h1, h2)
    max_w = max(w1, w2)
    
    # Pad image 1 to max H/W
    pad_h1 = max_h - h1
    pad_w1 = max_w - w1
    img1_padded = cv2.copyMakeBorder(img1, 0, pad_h1, 0, pad_w1, cv2.BORDER_CONSTANT, value=pad_val)
    
    # Pad image 2 to max H/W
    pad_h2 = max_h - h2
    pad_w2 = max_w - w2
    img2_padded = cv2.copyMakeBorder(img2, 0, pad_h2, 0, pad_w2, cv2.BORDER_CONSTANT, value=pad_val)
    
    return img1_padded, img2_padded

print("Manual MixUp/CutMix functions defined.")

Manual MixUp/CutMix functions defined.


In [8]:
# --- Cell 6: Define All 24 Augmentation Pipelines (Pure Albumentations) ---
# Each pipeline replicates the *full* transform chain from training.
# 'p=1.0' ensures the transform is applied for visualization.

# Common arguments for geometric transforms (fill with mean)
border_args = dict(border_mode=cv2.BORDER_CONSTANT, value=mean_val)

experiments = {
    # 1. Baseline: Resize only
    'exp01_baseline': A.Compose([resize_op]),
    
    # --- MMDET NATIVE REPLICATIONS (applied AFTER Resize) ---
    'exp02_hflip': A.Compose([resize_op, A.HorizontalFlip(p=1.0)]),
    'exp03_vflip': A.Compose([resize_op, A.VerticalFlip(p=1.0)]),
    'exp16_randomerasing': A.Compose([resize_op, A.CoarseDropout(max_holes=1, max_height=0.2, max_width=0.2, min_height=0.02, min_width=0.02, p=1.0, fill_value=mean_val)]),
    'exp18_gridmask': A.Compose([resize_op, A.GridDropout(p=1.0, fill_value=0)]),

    # --- ALBU REPLICATIONS (applied BEFORE Resize) ---
    'exp04_rotate': A.Compose([A.Rotate(limit=90, p=1.0, **border_args), resize_op]),
    
    # --- FIXED: Use A.RandomScale instead of A.Scale ---
    'exp05_scale': A.Compose([A.RandomScale(scale_limit=0.2, p=1.0, interpolation=cv2.INTER_LINEAR), resize_op]), 
    
    'exp06_affine': A.Compose([A.Affine(scale=0.2, translate_percent=0.1, rotate=15, shear=10, p=1.0, cval=mean_val[0], mode=cv2.BORDER_CONSTANT), resize_op]),
    'exp08_brightnesscontrast': A.Compose([A.RandomBrightnessContrast(p=1.0), resize_op]),
    'exp09_hsv': A.Compose([A.HueSaturationValue(p=1.0), resize_op]),
    'exp10_clahe': A.Compose([A.CLAHE(p=1.0), resize_op]),
    'exp11_channelshuffle': A.Compose([A.ChannelShuffle(p=1.0), resize_op]),
    'exp12_elastictransform': A.Compose([A.ElasticTransform(p=1.0, alpha=50, sigma=5, **border_args), resize_op]),
    'exp13_griddistortion': A.Compose([A.GridDistortion(p=1.0, **border_args), resize_op]),
    'exp14_opticaldistortion': A.Compose([A.OpticalDistortion(p=1.0, **border_args), resize_op]),
    'exp15_piecewiseaffine': A.Compose([A.PiecewiseAffine(p=1.0, cval=mean_val[0]), resize_op]),
    'exp17_cutout': A.Compose([A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=mean_val, p=1.0), resize_op]),
    'exp19_gaussnoise': A.Compose([A.GaussNoise(p=1.0), resize_op]),
    'exp20_motionblur': A.Compose([A.MotionBlur(blur_limit=(3, 11), p=1.0), resize_op]),
    'exp21_gaussianblur': A.Compose([A.GaussianBlur(blur_limit=(3, 7), p=1.0), resize_op]),
    'exp22_medianblur': A.Compose([A.MedianBlur(blur_limit=(3, 7), p=1.0), resize_op]),
    
    # --- SPECIAL CASE (REPLACES Resize) ---
    'exp07_randomresizedcrop': A.Compose([
        A.RandomResizedCrop(height=800, width=1333, scale=(0.5, 2.0), ratio=(0.75, 1.33), p=1.0)
    ]),
}

print(f"Defined {len(experiments)} Albumentations pipelines.")

Defined 22 Albumentations pipelines.


In [9]:
# --- Cell 7: Run Visualization (Standard Augmentations) ---

# This cell processes all pipelines defined in Cell 6
standard_experiments = {k: v for k, v in experiments.items()}

for exp_name, pipeline in tqdm(standard_experiments.items(), desc="Processing Experiments"):
    
    # Process all 25 images for this experiment
    for img_data in original_images:
        original_img = img_data['img'].copy()
        
        # Apply the full augmentation pipeline
        try:
            augmented_data = pipeline(image=original_img)
            augmented_img = augmented_data['image']
            
            # --- Save the comparison ---
            save_folder = os.path.join(output_dir, exp_name)
            save_filename = f"{os.path.splitext(img_data['filename'])[0]}_compare.jpg"
            
            create_and_save_comparison(original_img, augmented_img, save_folder, save_filename)

        except Exception as e:
            print(f"Error in pipeline {exp_name} for image {img_data['filename']}: {e}")

print("--- Standard augmentations complete ---")

Processing Experiments: 100%|██████████████████████████████████████████████████████████| 22/22 [00:25<00:00,  1.17s/it]

--- Standard augmentations complete ---


In [10]:
# --- Cell 8: Run Visualization (MixUp & CutMix) ---

# These two are special cases and use the manual functions from Cell 5
mosaic_experiments = [
    ('exp23_mixup', mixup_data),
    ('exp24_cutmix', cutmix_data)
]

for exp_name, transform_func in mosaic_experiments:
    print(f"--- Processing {exp_name} ---")
    
    for i in tqdm(range(len(original_images)), desc=f"Processing {exp_name}"):
        # Get image 1 and image 2
        img_data1 = original_images[i]
        # Get the next image (or loop back to the first for the last image)
        img_data2 = original_images[(i + 1) % len(original_images)] 

        original_img1 = img_data1['img'].copy()
        original_img2 = img_data2['img'].copy()

        # --- Apply the pipeline ---
        # 1. Resize both images first (replicating the training pipeline)
        img1_resized = resize_op(image=original_img1)['image']
        img2_resized = resize_op(image=original_img2)['image']
        
        # 2. Pad images to match max H/W (replicating batch padding)
        img1_padded, img2_padded = pad_to_match(img1_resized, img2_resized, mean_val)
        
        # 3. Apply the manual MixUp/CutMix function
        augmented_img = transform_func(img1_padded, img2_padded)

        # --- Save the comparison ---
        save_folder = os.path.join(output_dir, exp_name)
        save_filename = f"{os.path.splitext(img_data1['filename'])[0]}_compare.jpg"
        
        # Compare the *original* image 1 to the final mixed result
        create_and_save_comparison(original_img1, augmented_img, save_folder, save_filename)

print("--- MixUp/CutMix augmentations complete ---")

--- Processing exp23_mixup ---


Processing exp23_mixup: 100%|██████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 33.96it/s]


--- Processing exp24_cutmix ---


Processing exp24_cutmix: 100%|█████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 41.50it/s]

--- MixUp/CutMix augmentations complete ---
